In [1]:
%pip install transformers nltk


  Using cached transformers-4.48.1-py3-none-any.whl.metadata (44 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached filelock-3.17.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached huggingface_hub-0.28.0-py3-none-any.whl.metadata (13 kB)
  Using cached numpy-2.2.2-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.2-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
Using cached transformers-4.48.1-py3-none-any.whl (9.7 MB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached huggingface_hub-0.28.0-py3-none-any.whl (464 kB)
Using cached nump


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2

   ---------------------------------------- 0.0/172.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/172.3 MB 991.0 kB/s eta 0:02:54
   ---------------------------------------- 0.1/172.3 MB 1.1 MB/s eta 0:02:45
   ---------------------------------------- 0.2/172.3 MB 1.5 MB/s eta 0:01:57
   ---------------------------------------- 0.3/172.3 MB 2.1 MB/s eta 0:01:22
   ---------------------------------------- 0.7/172.3 MB 3.1 MB/s eta 0:00:55
   ---------------------------------------- 0.8/172.3 MB 3.3 MB/s eta 0:00:52
   ---------------------------------------- 0.9/172.3 MB 2.9 MB/s eta 0:01:01
   ---------------------------------------- 1.3/172.3 MB 3.8 MB/s eta 0:00:46
   ---------------------------------------- 1.4/172.3 MB 3.4 MB/s eta 0:00:51
   ---------------------------------------- 1.4/172.3 MB 3.4 MB/s eta 0:00:51
   ---------------------------------------- 1.4/172.3 MB 3.4 MB/s eta 0:00:51
   ---------------------------------------- 1.4/172.3 MB 3.4 MB/s eta


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from web3 import Web3
import json

In [4]:
from web3 import Web3

# Connect to the local Ganache blockchain
ganache_url = "http://127.0.0.1:7545"  # Default Ganache RPC URL
web3 = Web3(Web3.HTTPProvider(ganache_url))

# Check the connection
if web3.is_connected():
    print("Connected to the Ganache blockchain")
else:
    print("Failed to connect")

Connected to the Ganache blockchain


In [5]:
# Deployed contract details
contract_address = "0xAfE36632F6D75A02eE4D83c9Cd4D2650880aE691"  # Replace with your deployed contract address
with open("Healthcare.abi.json", "r") as abi_file:  # Save the ABI from the previous step as a JSON file
    abi = json.load(abi_file)
healthcare_contract = web3.eth.contract(address=contract_address, abi=abi)

# Set default account
web3.eth.default_account = web3.eth.accounts[0]

In [6]:
from transformers import pipeline

# AI Setup: Load a pretrained model for summarization
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device="cpu")

k:\Experiment\blockchain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "k:\Experiment\blockchain\.venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "k:\Experiment\blockchain\.ven

In [7]:
# Add text data to the blockchain
def add_health_record(name, text_data):
    tx_hash = healthcare_contract.functions.addRecord(name, text_data).transact()
    web3.eth.wait_for_transaction_receipt(tx_hash)
    print(f"Record for {name} added to blockchain.")

# Retrieve and analyze health data
def analyze_health_record(record_id):
    # Fetch the record from the blockchain
    record = healthcare_contract.functions.getRecord(record_id).call()
    name, health_data, owner = record

    print(f"Analyzing Record ID: {record_id}")
    print(f"Patient Name: {name}")
    print(f"Owner Address: {owner}")
    print(f"Original Text Data: {health_data}")

    # Summarize the text data
    summary = summarizer(health_data, max_length=50, min_length=10, do_sample=False)
    print(f"Summarized Data: {summary[0]['summary_text']}")

In [8]:
# Example Usage
# 1. Add a record
add_health_record("John Doe", "Patient shows symptoms of severe headache and fatigue. History of high blood pressure.")

# 2. Analyze the record
analyze_health_record(1)


Your max_length is set to 50, but your input_length is only 6. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=3)


Record for John Doe added to blockchain.
Analyzing Record ID: 1
Patient Name: John Doe
Owner Address: 0x42F78a2db8619B47F1A4c280DEc19519AA969eb1
Original Text Data: EncryptedData123
Summarized Data:  encrypts all data in a secure way. Encrypted data can only be decrypted with a password.


In [9]:
%pip install cryptography

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from cryptography.fernet import Fernet

# Check if key already exists in previous cells, if not generate new one
if 'encryption_key' not in globals():
    encryption_key = Fernet.generate_key()
    print("Generated new encryption key")
else:
    print("Using existing encryption key")

cipher = Fernet(encryption_key)

# Encrypt and decrypt functions
def encrypt_data(data):
    return cipher.encrypt(data.encode()).decode()

def decrypt_data(encrypted_data):
    return cipher.decrypt(encrypted_data.encode()).decode()


Using existing encryption key


In [14]:
# Example Usage
encrypted_text = encrypt_data("Patient exhibits mild fever and cold symptoms.")
try:
    # Store encrypted data in the blockchain
    add_health_record("Jane Doe", encrypted_text)

    # Retrieve and decrypt for analysis
    record = healthcare_contract.functions.getRecord(2).call()
    decrypted_health_data = decrypt_data(record[1])
    print("Decrypted Record Data:", decrypted_health_data)
    
    # Optional: Analyze the decrypted data
    summary = summarizer(decrypted_health_data, max_length=50, min_length=10, do_sample=False)
    print(f"AI Summary: {summary[0]['summary_text']}")
    
except Exception as e:
    print(f"Error occurred: {str(e)}")
    print("Make sure you're using the correct encryption key and the record exists.")

Record for Jane Doe added to blockchain.
Error occurred: 
Make sure you're using the correct encryption key and the record exists.
